The following setup was copied and pasted from the Starting Points blog post.
It sets up the domain of individuals, the valuation function that
holds the information about the properties of the individuals in the model,
and sets up a valuation function.
It also prints "Setting up the world." for no particularly good reason.

In [1]:
import nltk
print("Setting up the world.")
squares = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']
dom = {'a', 'b', 'c', 'd', 'e'} | set(squares)
valstr = """
square => {s1, s2, s3, s4, s5, s6, s7, s8}
odd => {s1, s3, s5, s7}
even => {s2, s4, s6, s8}
block => {a, b}
pyramid => {c, e}
table => {d}
thing => {a, b, c, d, e}
red => {a}
blue => {b, e}
green => {c, d}
on => {(a,s1),(b,s2),(d,s4),(c,d)}
"""
val = nltk.sem.Valuation.fromstring(valstr)
val['held'] = {('e',)}
m = nltk.Model(dom, val)
g = nltk.Assignment(dom)

Setting up the world.


Just to make sure it is working, we can look at what our valuation function returns for "square". Though we also have a list of the squares in a somewhat more usable form, set up above.

In [2]:
val['square']

{('s1',), ('s2',), ('s3',), ('s4',), ('s5',), ('s6',), ('s7',), ('s8',)}

In [3]:
squares

['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']

See the homework for a description of what we're doing in more detail, but: the plot is that we want to build up a way to draw the world. We're going to use text, not graphics. So we are going to need to draw it left-to-right and top-to-bottom. A prerequisite for this is knowing what is on each square, so we are going to need to make a list of the "stacks" (the things that are stacked on each square, in order, which we are going to need to draw).

We are going to want to create a list of stacks, one per square, and that's what the list comprehension two down does.  It relies on the ability to build a stack, but we're postponing that for a bit.  In the meanwhile, we'll define a "placeholder" function `build_stack(s)`, but we will revise that later to actually do something. We're breaking the problem down into smaller problems.

In [4]:
def build_stack(s):
    return False

In [5]:
stacks = [build_stack(s) for s in squares]

Now, we want to write `build_stack(s)` but in order to know what is in a stack, we need to be able to figure out (given our model of the world) what is on an object.  Like, for example, what is on square `s1`.

The way we'll do that is to create a formula/expression `f` that represents `on(x,s)`.  That is, an expression that is true if `x` is on `s` in the world.  So far, it's very abstract.  But then we define an assignment such that `s` points to the object `obj` we are inquiring about (the thing that was passed in as an argument).  In the context of our expression `f`, this is something like evaluating as meaning "the `x` that is on *that* (pointing to `obj`)".

Then, we ask what values of `x` produce truth in our model `m` under the (pointing) assignment `g2`.  We are actually making an assumption here that there is either going to be one such `x` or zero such `x`es.  If there is one, `next_obj` will be that object (that is on `obj`).  If there isn't one, the line under `try:` will result in an error (an attempt to look at the first element of an empty list), which will cause the `except:` block to be used, and in that case `next_obj` will be assigned `None`.

In [6]:
def whats_on(obj):
    f = nltk.sem.Expression.fromstring("on(x,s)")
    g2 = nltk.Assignment(dom, [('s', obj)])
    try:
        next_obj = list(m.satisfiers(f, 'x', g2))[0]
    except:
        next_obj = None
    return next_obj

In [7]:
print(whats_on('s7'))

None


In [8]:
print(whats_on('s4'))

d


In [9]:
print(whats_on('d'))

c


In [10]:
print(whats_on('c'))

None


Brief tangent about what a `while` block is.  Normally, you say `while <condition>:` and go through the block until `<condition>` becomes false.  For example in the counting code below.  (Also `x += 1` is a kind of shorthand for `x = x + 1`.)

In [11]:
x = 0
while x < 4:
    print(x)
    x += 1


0
1
2
3


In our version of `build_stack(square)` below, we make use of `while:` to keep looking at what's on top of the next thing until there are no more things.

In [12]:
def build_stack(square):
    stack = [square]
    while whats_on(stack[-1]):
        next_obj = whats_on(stack[-1])
        stack.append(next_obj)
    return stack
        

In [13]:
build_stack('s4')

['s4', 'd', 'c']

In the homework write-up there is a different version of this.  It is either more or less elegant, it depends.  The advantage of the one from the homework below is that it only calls upon `whats_on()` once per loop, so if it were costly or resource-intensive to execute `whats_on()` then the one below is better, even if the structure of the `while:` block is more inelegant.  What is happening below is that the `while True:` essentially initiates an infinite loop, which we `break` out of when we're done.

In [14]:
def build_stack_hw(square):
    stack = [square]
    while True:
        next_obj = whats_on(stack[-1])
        if next_obj:
            stack.append(next_obj)
        else:
            break
    return stack

In [15]:
build_stack_hw('s4')

['s4', 'd', 'c']

Let's see what stacks we get.  This is a little function that will build them.  I no longer remember why I defined a function that we will not use again, but anyway.

In [16]:
def the_stacks():
    return [build_stack(s) for s in squares]

In [17]:
the_stacks()

[['s1', 'a'],
 ['s2', 'b'],
 ['s3'],
 ['s4', 'd', 'c'],
 ['s5'],
 ['s6'],
 ['s7'],
 ['s8']]

Now, for each shape we will need to draw it.  There's a bit of thought that went into determining how this was going to work.  The conclusion is that we are going to make each shape 8 characters wide and 4 characters tall, with an ASCII-art style border and a label.  We are going to need to print these out strategically, so the first step is to collect the 4 lines of 8 characters.  The following is the proof of concept.  The `[1:-1]` at the end is to remove the superfluous empty lines that we get from splitting on the linefeed `\n` character, and the format string `{: ^6}` instructs the label to print centered, 6 characters wide, padded with spaces.

In [18]:
def draw_shape(obj):
    s = r"""
/------\
|      |
|{: ^6}|
\------/
""".format('label!')
    return s.split("\n")[1:-1]

In [19]:
draw_shape('s')

['/------\\', '|      |', '|label!|', '\\------/']

In [20]:
print("\n".join(draw_shape('a')))

/------\
|      |
|label!|
\------/


When we want to draw a more general shape we want its label to represent something about its properties, like its color.  So we want to have a way to get an object's properties.  We can consult the valuation function on that, to ask what predicates have this object within them.

In [21]:
def obj_properties(obj):
    return {v for v in val if (obj,) in val[v]}

In [22]:
obj_properties('s1')

{'odd', 'square'}

The following was copied from the "Starting points II" blog post.  It takes an object, finds its properties, extracts out its shape, ignores the predicates "held" and "thing", and then builds a string (for a label) out of the first 2 characters of each property (apart from shape, held, and thing) the object has.  When we are about to draw an object, we will call `obj_form()` to find out what shape to draw and what label to apply.

In [23]:
def obj_form(obj):
    properties = obj_properties(obj) - {'held', 'thing'}
    shape = {'block', 'pyramid', 'table', 'square'} & properties
    abbrevs = [prop[:2] for prop in properties - shape]
    label = "".join(abbrevs)[:6]
    return(shape.pop(), label)

Square `s1` is a square with the property `odd`, so the label is `"od"`.

In [24]:
obj_form('s1')

('square', 'od')

Copied from the "Starting points II" blog post. This draws a hand if asked to draw a hand, otherwise gets the shape and label using `obj_form()` and then uses multiple `if` statements to determine which shape to embed the label in.  If it's not a block, pyramid, or table, then it assumes we are looking at a square.  This is basically just a generalization of the proof of concept `draw_shape()` defined earlier above.

In [25]:
def draw_shape(obj):
    if obj:
        if obj == "HAND":
            s = r"""
       /
======<=
       \
        
"""
        else:
            (shape, label) = obj_form(obj)
            if shape == 'block':
                s = r"""
/------\
|      |
|{: ^6}|
\------/
""".format(label)
            elif shape == 'pyramid':
                s = r"""
   /\   
  /  \  
 /    \ 
/{:_^6}\
""".format(label)
            elif shape == 'table':
                s = r"""
|------|
|      |
|{: ^6}|
|      |
""".format(label)
            else:
                s = r"""
========
|======|
|{:=^6}|
|======|
""".format(label)
        return s.split("\n")[1:-1]
    else:
        return [' '*8]*4

In [26]:
draw_shape('a')

['/------\\', '|      |', '|  re  |', '\\------/']

In [27]:
draw_shape('b')

['/------\\', '|      |', '|  bl  |', '\\------/']

In [28]:
draw_shape(None)

['        ', '        ', '        ', '        ']

Side note on how the `None` shape is constructed.  You can multiply strings, you can multiply lists, and since what we want is a list of 4 strings of 8 spaces, there's a kind of cutesy way you can write that.

In [29]:
'hello' * 4

'hellohellohellohello'

In [30]:
[' '*8]*4

['        ', '        ', '        ', '        ']

Figuring out what has the "held" property (which in this model represents what the robot is holding).

In [31]:
val['held']

{('e',)}

In [32]:
held_thing = list(val['held'])

In [33]:
held_thing

[('e',)]

In [34]:
held_thing[0]

('e',)

In [35]:
held_thing[0][0]

'e'

Formalizing it in a function that can just return to us the object being held.

In [36]:
def obj_in_hand():
    if len(val['held']) == 0:
        return None
    else:
        return list(val['held'])[0][0]

In [37]:
obj_in_hand()

'e'

Copied from the "Starting points II" blog post, and we walked through how it works.  But it's worth thinking through it too.  This is the main world-draw-er.

In [38]:
def draw(m, g):
    stacks = [build_stack(square) for square in squares]
    # build up from the bottom
    rows = []
    while True:
        current_row = len(rows)
        empty = True
        row = []
        for stack in stacks:
            if current_row < len(stack):
                row.append(draw_shape(stack[current_row]))
                empty = False
            else:
                row.append(draw_shape(None))
        if empty:
            break
        rows.append(row)
    rows.append([[' ',' ']])
    rows.append([draw_shape('HAND'), draw_shape(obj_in_hand())])
    for row in reversed(rows):
        for line in range(len(row[0])):
            print("".join([col[line] for col in row]))

In [39]:
draw(m, g)

       /   /\   
======<=  /  \  
       \ /    \ 
        /__bl__\
 
 
                           /\                                   
                          /  \                                  
                         /    \                                 
                        /__gr__\                                
/------\/------\        |------|                                
|      ||      |        |      |                                
|  re  ||  bl  |        |  gr  |                                
\------/\------/        |      |                                
|======||======||======||======||======||======||======||======|
|==od==||==ev==||==od==||==ev==||==od==||==ev==||==od==||==ev==|
|======||======||======||======||======||======||======||======|


This is general, it has drawn the world.  We can change the world and the representation will change.  For example: The things that are blue are b and e.  We can add c to this (which was green before), meaning that now c is both blue and green.  And, lo!  The pyramid at the top of the tall stack now gets a label "grbl", representing the fact that it has both green and blue properties.

In [40]:
val['blue']

{('b',), ('e',)}

In [41]:
val['blue'].update({('c',)})

In [42]:
val['blue']

{('b',), ('c',), ('e',)}

In [43]:
draw(m, g)

       /   /\   
======<=  /  \  
       \ /    \ 
        /__bl__\
 
 
                           /\                                   
                          /  \                                  
                         /    \                                 
                        /_grbl_\                                
/------\/------\        |------|                                
|      ||      |        |      |                                
|  re  ||  bl  |        |  gr  |                                
\------/\------/        |      |                                
|======||======||======||======||======||======||======||======|
|==od==||==ev==||==od==||==ev==||==od==||==ev==||==od==||==ev==|
|======||======||======||======||======||======||======||======|


We're now basically done with the drawing part, so now we can move to the language parsing part.  We're going to build up a context free grammar with feature passing.  And we're going to try to automate some of this because we can.

In [45]:
from nltk import grammar

In [44]:
nouns = {'block', 'pyramid', 'table', 'square', 'thing'}
adjectives = {'red', 'blue', 'green', 'white', 'big', 'small', 'odd', 'even'}
prepositions = {'on'}

All of the noun rules will look like this, and we need one per noun.

In [46]:
npstrblock = r"""
NP[SEM=<\x.block(x)>] -> 'block'
"""

So, here's something that will make a line if given a noun.  Note the named replacements in the `.format()` string too, allowing the noun's name to be used twice.

In [47]:
def npstring(noun):
    return r"NP[SEM=<\x.{n}(x)>] -> '{n}'".format(n=noun)

We get what we wanted for "block"

In [48]:
print(npstring('block'))

NP[SEM=<\x.block(x)>] -> 'block'


So, now, let's build a string with all of the noun definitions.

In [52]:
npdef = "\n".join([npstring(noun) for noun in nouns])

In [53]:
print(npdef)

NP[SEM=<\x.table(x)>] -> 'table'
NP[SEM=<\x.pyramid(x)>] -> 'pyramid'
NP[SEM=<\x.square(x)>] -> 'square'
NP[SEM=<\x.block(x)>] -> 'block'
NP[SEM=<\x.thing(x)>] -> 'thing'


And, same for adjectives.  They're just like nouns.  Prepositions are like transitive verbs. In class we pondered a bit the definition of "on" there, but the secret to its complexity is the fact that names are treated as having the same type as quantifiers like "every block".  It winds up meaning that names are represented not as individuals but as the colleciton of properties the individual has.  See the end of homework 7 and the discussion there.

In [49]:
def adjstring(adj):
    return r"Adj[SEM=<\x.{a}(x)>] -> '{a}'".format(a=adj)

In [50]:
def pstring(prep):
    return r"P[SEM=<\X x.X(\y.{p}(x,y))>] -> '{p}'".format(p=prep)

In [51]:
pstring('on')

"P[SEM=<\\X x.X(\\y.on(x,y))>] -> 'on'"

In [54]:
adjdef = "\n".join([adjstring(adj) for adj in adjectives])
pdef = "\n".join([pstring(prep) for prep in prepositions])

We define structural composition rules that allow us to build a DP from a Det and an NP, and we define a recursive NP rule that allow us to build Adj-N structures.  Here we are actually starting to compose semantic values.  Below, we have a `DEF` feature that is being kept track of as well, separate from the `SEM` feature.  A DP will be `+DEF` iff its Det is `+DEF`, and the semantic value of the DP is the result of giving the semantic value of the NP as an argument to the semantic value of Det.  For adjectives, we presume that Adj+NP is true of something that is both Adj and NP.  (This works for "intersective" adjectives like "green".  It wouldn't work well for something like "tall" or "fake" but that's for some other day.)

In [55]:
dpdef = r"""
DP[SEM=<?det(?np)>, DEF=?def] -> Det[SEM=?det, DEF=?def] NP[SEM=?np]
NP[SEM=<\x.(?adj(x) & ?np(x))>] -> Adj[SEM=?adj] NP[SEM=?np]
"""

Our determiners are quantifiers, a(n), the, and every.  We're treating "the" here as basically being "a" except with a `+DEF` feature that, at some point, we might make use of.

In [56]:
detdef = r"""
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>, -DEF] -> 'a'
Det[SEM=<\P Q.all x.(P(x) -> Q(x))>, -DEF] -> 'every'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>, -DEF] -> 'an'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>, +DEF] -> 'the'
"""

A PP is pretty much like a VP, the P is a function that takes the object as an argument.

In [57]:
ppdef = r"""
PP[SEM=<?p(?dp)>] -> P[SEM=?p] DP[SEM=?dp]
"""

At the moment the only kind of predicate we have is the P "on".  So, we will add a meaningless copula verb to make a VP out of a PP.  The semantics of the PP are just passed up unchanged.

In [58]:
vpdef = r"""
VCOP -> 'is'
VP[SEM=?pp] -> VCOP PP[SEM=?pp]
"""

And then the top level.  I included CP, CBAR, as well as S, but we certainly didn't use this in class.  It will be of some use later.  The CT feature stands for "clause type" and the one sentence type we have so far is the declarative (`CT='dec'`) type.  CP and CBAR don't do anything except pass the semantics up unchanged.  The semantic value of an S is the semantic value of the subject (which, recall, is a high type like for "every block") to the semantic value of the predicate.

In [59]:
sdef = r"""
% start CP
S[SEM=<?subj(?vp)>, CT='dec'] -> DP[SEM=?subj] VP[SEM=?vp]
CBAR[SEM=?s, CT=?ct] -> S[SEM=?s, CT=?ct]
CP[SEM=?cbar, CT=?ct] -> CBAR[SEM=?cbar, CT=?ct]
"""

Last thing, as we hit the end of class time.  Put the context free grammar together.

In [60]:
cfgdef = "\n".join([sdef, vpdef, adjdef, npdef, dpdef, ppdef, pdef, detdef])

In [61]:
print(cfgdef)


% start CP
S[SEM=<?subj(?vp)>, CT='dec'] -> DP[SEM=?subj] VP[SEM=?vp]
CBAR[SEM=?s, CT=?ct] -> S[SEM=?s, CT=?ct]
CP[SEM=?cbar, CT=?ct] -> CBAR[SEM=?cbar, CT=?ct]


VCOP -> 'is'
VP[SEM=?pp] -> VCOP PP[SEM=?pp]

Adj[SEM=<\x.blue(x)>] -> 'blue'
Adj[SEM=<\x.small(x)>] -> 'small'
Adj[SEM=<\x.big(x)>] -> 'big'
Adj[SEM=<\x.green(x)>] -> 'green'
Adj[SEM=<\x.white(x)>] -> 'white'
Adj[SEM=<\x.even(x)>] -> 'even'
Adj[SEM=<\x.odd(x)>] -> 'odd'
Adj[SEM=<\x.red(x)>] -> 'red'
NP[SEM=<\x.table(x)>] -> 'table'
NP[SEM=<\x.pyramid(x)>] -> 'pyramid'
NP[SEM=<\x.square(x)>] -> 'square'
NP[SEM=<\x.block(x)>] -> 'block'
NP[SEM=<\x.thing(x)>] -> 'thing'

DP[SEM=<?det(?np)>, DEF=?def] -> Det[SEM=?det, DEF=?def] NP[SEM=?np]
NP[SEM=<\x.(?adj(x) & ?np(x))>] -> Adj[SEM=?adj] NP[SEM=?np]


PP[SEM=<?p(?dp)>] -> P[SEM=?p] DP[SEM=?dp]

P[SEM=<\X x.X(\y.on(x,y))>] -> 'on'

Det[SEM=<\P Q.exists x.(P(x) & Q(x))>, -DEF] -> 'a'
Det[SEM=<\P Q.all x.(P(x) -> Q(x))>, -DEF] -> 'every'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>, -DEF

Define a feature grammar from the definition above.

In [62]:
gram = grammar.FeatureGrammar.fromstring(cfgdef)

Define a parser useing that grammar.

In [63]:
cp = nltk.FeatureChartParser(gram)

Define a sentence that we are going to test.

In [64]:
sent = 'a block is on the table'

What are the parses of that sentence given our grammar?

In [65]:
parses = list(cp.parse(sent.split()))

Show me the first (only) parse.

In [66]:
print(parses[0])

(CP[CT='dec', SEM=<exists x.(block(x) & exists z1.(table(z1) & on(x,z1)))>]
  (CBAR[CT='dec', SEM=<exists x.(block(x) & exists z1.(table(z1) & on(x,z1)))>]
    (S[CT='dec', SEM=<exists x.(block(x) & exists z1.(table(z1) & on(x,z1)))>]
      (DP[-DEF, SEM=<\Q.exists x.(block(x) & Q(x))>]
        (Det[-DEF, SEM=<\P Q.exists x.(P(x) & Q(x))>] a)
        (NP[SEM=<\x.block(x)>] block))
      (VP[SEM=<\x.exists z1.(table(z1) & on(x,z1))>]
        (VCOP[] is)
        (PP[SEM=<\x.exists z1.(table(z1) & on(x,z1))>]
          (P[SEM=<\X x.X(\y.on(x,y))>] on)
          (DP[+DEF, SEM=<\Q.exists x.(table(x) & Q(x))>]
            (Det[+DEF, SEM=<\P Q.exists x.(P(x) & Q(x))>] the)
            (NP[SEM=<\x.table(x)>] table)))))))


What is the semantic value of the top node?  (In other words, what are the truth conditions of this sentence?)

In [67]:
print(parses[0].label()['SEM'])

exists x.(block(x) & exists z1.(table(z1) & on(x,z1)))


That's what we wanted, if you work through it.  There is an x such that x is a block and there is also a z such that z is a table, and x is on z.  That is: "a block is on a/the table."